In [26]:
import pandas as pd
from IPython.display import display as dis
import requests as req
from bs4 import BeautifulSoup as bs
import re
import numpy as np
from itertools import combinations
from tqdm import tqdm_notebook

In [40]:
legs = pd.read_excel(r"C:\Users\user\Documents\Coding\My Files\pokemon project\legendary and mythical pokemon.xlsx")
legs = [x for list in legs.values.tolist() for x in list if x != False]

type_chart = pd.read_excel(r"C:\Users\user\Documents\Coding\My Files\pokemon project\type_chart.xlsx")
type_chart = type_chart.set_index('attack_type')

pok_df = pd.read_excel(r"C:\Users\user\Documents\Coding\My Files\pokemon project\gen 4 pokemon psypokes.xlsx")

In [41]:
pok_df['type'] = np.nan
pok_df['type'], pok_df['type1'], pok_df['type2'] = pok_df['type'].astype(str), pok_df['type1'].astype(str), pok_df['type2'].astype(str)

for i, pok in enumerate(pok_df.itertuples()):
    if pok.type2 == str(0):
         pok_df.at[i, 'type'] = pok.type1
    else:
        pok_df.at[i, 'type'] = frozenset([pok.type1, pok.type2])

pok_df


,rank,pokemon,hp,atk,def,satk,sdef,spd,type1,type2,total,alttotal,type
0,13,Slaking,150,160,100,95,65,100,normal,0,670,575,normal
1,21,Dragonite,91,134,95,100,100,80,dragon,flying,600,500,"(flying, dragon)"
2,22,Garchomp,108,130,95,80,85,102,dragon,ground,600,520,"(ground, dragon)"
3,28,Metagross,80,135,130,95,90,70,steel,psychic,600,505,"(steel, psychic)"
4,30,Salamence,95,135,80,110,80,100,dragon,flying,600,490,"(flying, dragon)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,504,Weedle,40,35,30,20,20,50,bug,poison,195,175,"(bug, poison)"
464,505,Wurmple,45,45,35,20,30,20,bug,0,195,175,bug
465,506,Kricketot,37,25,41,25,41,25,bug,0,194,169,bug
466,507,Azurill,50,20,40,20,40,20,normal,0,190,170,normal


In [42]:
pok_df = pok_df[~pok_df['pokemon'].isin(["Swampert","Charizard","Gardevoir","Clefable"])]
pok_df

,rank,pokemon,hp,atk,def,satk,sdef,spd,type1,type2,total,alttotal,type
0,13,Slaking,150,160,100,95,65,100,normal,0,670,575,normal
1,21,Dragonite,91,134,95,100,100,80,dragon,flying,600,500,"(flying, dragon)"
2,22,Garchomp,108,130,95,80,85,102,dragon,ground,600,520,"(ground, dragon)"
3,28,Metagross,80,135,130,95,90,70,steel,psychic,600,505,"(steel, psychic)"
4,30,Salamence,95,135,80,110,80,100,dragon,flying,600,490,"(flying, dragon)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,504,Weedle,40,35,30,20,20,50,bug,poison,195,175,"(bug, poison)"
464,505,Wurmple,45,45,35,20,30,20,bug,0,195,175,bug
465,506,Kricketot,37,25,41,25,41,25,bug,0,194,169,bug
466,507,Azurill,50,20,40,20,40,20,normal,0,190,170,normal


In [43]:
type_chart = type_chart.drop('fairy').drop('fairy',axis=1)
type_chart

,normal,fire,water,electric,grass,ice,fighting,poison,ground,flying,psychic,bug,rock,ghost,dragon,dark,steel
attack_type,,,,,,,,,,,,,,,,,
normal,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.0,1.0,1.0,0.5
fire,1,0.5,0.5,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,0.5,1.0,0.5,1.0,2.0
water,1,2.0,0.5,1.0,0.5,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0,1.0
electric,1,1.0,2.0,0.5,0.5,1.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0
grass,1,0.5,2.0,1.0,0.5,1.0,1.0,0.5,2.0,0.5,1.0,0.5,2.0,1.0,0.5,1.0,0.5
ice,1,0.5,0.5,1.0,2.0,0.5,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,0.5
fighting,2,1.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0,0.5,0.5,0.5,2.0,0.0,1.0,2.0,2.0
poison,1,1.0,1.0,1.0,2.0,1.0,1.0,0.5,0.5,1.0,1.0,1.0,0.5,0.5,1.0,1.0,0.0
ground,1,2.0,1.0,2.0,0.5,1.0,1.0,2.0,1.0,0.0,1.0,0.5,2.0,1.0,1.0,1.0,2.0


In [44]:
two_types = pok_df['type'][pok_df['type'].apply(lambda x: isinstance(x, frozenset))]
two_types = pd.DataFrame(two_types).set_index(['type'])
two_types = two_types[~two_types.index.duplicated(keep='first')]
two_types*two_types.T

type,"(flying, dragon)","(ground, dragon)","(steel, psychic)","(rock, dark)","(flying, normal)","(flying, water)","(dragon, water)","(poison, flying)","(ice, water)","(steel, electric)",...,"(psychic, ice)","(bug, poison)","(psychic, rock)","(bug, grass)","(normal, water)","(fire, rock)","(flying, ice)","(bug, water)","(bug, ground)","(bug, ghost)"
type,,,,,,,,,,,,,,,,,,,,,
"(flying, dragon)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(ground, dragon)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(steel, psychic)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(rock, dark)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(flying, normal)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(fire, rock)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(flying, ice)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(bug, water)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
type_chart_joined = pd.concat([type_chart, two_types], axis=0, join='outer')
type_chart_joined = pd.concat([type_chart_joined.T, two_types], axis=1, join='outer')
type_chart_joined = type_chart_joined.T


In [46]:
for type, row in type_chart_joined.T.iterrows():
    types = [list(x) for x in row.index if isinstance(x, frozenset)]
    for line in types:
        value = max(row[line[0]], row[line[1]])
        type_chart_joined.at[frozenset(line), type] = value

type_chart_joined

,normal,fire,water,electric,grass,ice,fighting,poison,ground,flying,...,"(psychic, ice)","(bug, poison)","(psychic, rock)","(bug, grass)","(normal, water)","(fire, rock)","(flying, ice)","(bug, water)","(bug, ground)","(bug, ghost)"
normal,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fire,1.0,0.5,0.5,1.0,2.0,2.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
water,1.0,2.0,0.5,1.0,0.5,1.0,1.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
electric,1.0,1.0,2.0,0.5,0.5,1.0,1.0,1.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
grass,1.0,0.5,2.0,1.0,0.5,1.0,1.0,0.5,2.0,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(fire, rock)",1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(flying, ice)",1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(bug, water)",1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(bug, ground)",1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
for type, row in type_chart_joined.iterrows():
    d_types = [list(x) for x in row.index if isinstance(x, frozenset)]
    if not isinstance(type, frozenset):
        for c_type in d_types:
            type_chart_joined.at[type, frozenset(c_type)] = row[c_type[0]]*row[c_type[1]]
    else:
        for c_type in d_types:
            r_type = list(type)
            type_chart_joined.at[type, frozenset(c_type)] = max(type_chart_joined.loc[r_type[0], c_type[0]]*type_chart_joined.loc[r_type[0], c_type[1]], 
            type_chart_joined.loc[r_type[1], c_type[0]]*type_chart_joined.loc[r_type[1], c_type[1]])

type_chart_joined

,normal,fire,water,electric,grass,ice,fighting,poison,ground,flying,...,"(psychic, ice)","(bug, poison)","(psychic, rock)","(bug, grass)","(normal, water)","(fire, rock)","(flying, ice)","(bug, water)","(bug, ground)","(bug, ghost)"
normal,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.00,0.5,1.00,1.0,0.50,1.0,1.0,1.0,0.0
fire,1.0,0.5,0.5,1.0,2.0,2.0,1.0,1.0,1.0,1.0,...,2.0,2.00,0.5,4.00,0.5,0.25,2.0,1.0,2.0,2.0
water,1.0,2.0,0.5,1.0,0.5,1.0,1.0,1.0,2.0,1.0,...,1.0,1.00,2.0,0.50,0.5,4.00,1.0,0.5,2.0,1.0
electric,1.0,1.0,2.0,0.5,0.5,1.0,1.0,1.0,0.0,2.0,...,1.0,1.00,1.0,0.50,2.0,1.00,2.0,2.0,0.0,1.0
grass,1.0,0.5,2.0,1.0,0.5,1.0,1.0,0.5,2.0,0.5,...,1.0,0.25,2.0,0.25,2.0,1.00,0.5,1.0,1.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(fire, rock)",1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,...,2.0,2.00,1.0,4.00,1.0,2.00,4.0,2.0,2.0,2.0
"(flying, ice)",1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,...,1.0,2.00,1.0,4.00,1.0,0.50,1.0,2.0,2.0,2.0
"(bug, water)",1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,...,2.0,1.00,2.0,2.00,1.0,4.00,1.0,1.0,2.0,1.0
"(bug, ground)",1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,0.5,...,2.0,1.00,2.0,2.00,1.0,4.00,0.5,1.0,1.0,0.5


In [48]:
def func(a,b):
    if a <= 1 and b < 1:
        return a
    elif b == 0:
        return 4
    else:
        return a/b

vfunc = np.vectorize(func)

scores = {}
for index, row in type_chart_joined.iterrows():
    attack = row.values
    defense = type_chart_joined[index].values
    score = vfunc(attack, defense)
    #score = np.where(attack <= 1 and defense < 1, attack, attack/defense)
    scores[index] = score

#     n_a = df['final_score'].values

# Vectorize your function

# vfunc = np.vectorize(my_function)

# Calculate the result array using vectorized function:

# res_array = vfunc(n_a)

# df['final_score'] = res_array

In [49]:
scores_df = pd.DataFrame.from_dict(scores).T
scores_df.columns = scores_df.index
scores_df

,normal,fire,water,electric,grass,ice,fighting,poison,ground,flying,...,"(psychic, ice)","(bug, poison)","(psychic, rock)","(bug, grass)","(normal, water)","(fire, rock)","(flying, ice)","(bug, water)","(bug, ground)","(bug, ghost)"
normal,1.0,1.00,1.00,1.0,1.00,1.0,0.5,1.00,1.00,1.00,...,1.0,1.000,0.50,1.000,1.00,0.500,1.00,1.0,1.0,0.00
fire,1.0,0.50,0.25,1.0,4.00,4.0,1.0,1.00,0.50,1.00,...,2.0,2.000,0.25,8.000,0.25,0.125,2.00,0.5,1.0,2.00
water,1.0,4.00,0.50,0.5,0.25,1.0,1.0,1.00,2.00,1.00,...,1.0,1.000,2.00,0.250,0.50,4.000,1.00,0.5,2.0,1.00
electric,1.0,1.00,2.00,0.5,0.50,1.0,1.0,1.00,0.00,4.00,...,1.0,1.000,1.00,0.500,2.00,1.000,2.00,2.0,0.0,1.00
grass,1.0,0.25,4.00,1.0,0.50,0.5,1.0,0.25,4.00,0.25,...,0.5,0.125,2.00,0.125,2.00,0.500,0.25,0.5,0.5,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(fire, rock)",1.0,8.00,0.25,1.0,2.00,4.0,0.5,1.00,0.25,4.00,...,2.0,4.000,0.50,4.000,0.25,1.000,8.00,0.5,0.5,2.00
"(flying, ice)",1.0,0.50,1.00,0.5,4.00,1.0,2.0,1.00,4.00,2.00,...,1.0,2.000,0.25,8.000,1.00,0.125,1.00,2.0,4.0,2.00
"(bug, water)",1.0,2.00,1.00,0.5,2.00,1.0,1.0,1.00,4.00,0.50,...,2.0,1.000,1.00,2.000,1.00,2.000,0.50,1.0,2.0,1.00
"(bug, ground)",1.0,1.00,0.50,4.0,2.00,0.5,1.0,4.00,1.00,0.25,...,1.0,1.000,2.00,2.000,0.50,2.000,0.25,0.5,1.0,0.50


In [50]:
scores_df = scores_df.fillna(0)
scores_df = scores_df.replace(np.inf, 4)
scores_df


,normal,fire,water,electric,grass,ice,fighting,poison,ground,flying,...,"(psychic, ice)","(bug, poison)","(psychic, rock)","(bug, grass)","(normal, water)","(fire, rock)","(flying, ice)","(bug, water)","(bug, ground)","(bug, ghost)"
normal,1.0,1.00,1.00,1.0,1.00,1.0,0.5,1.00,1.00,1.00,...,1.0,1.000,0.50,1.000,1.00,0.500,1.00,1.0,1.0,0.00
fire,1.0,0.50,0.25,1.0,4.00,4.0,1.0,1.00,0.50,1.00,...,2.0,2.000,0.25,8.000,0.25,0.125,2.00,0.5,1.0,2.00
water,1.0,4.00,0.50,0.5,0.25,1.0,1.0,1.00,2.00,1.00,...,1.0,1.000,2.00,0.250,0.50,4.000,1.00,0.5,2.0,1.00
electric,1.0,1.00,2.00,0.5,0.50,1.0,1.0,1.00,0.00,4.00,...,1.0,1.000,1.00,0.500,2.00,1.000,2.00,2.0,0.0,1.00
grass,1.0,0.25,4.00,1.0,0.50,0.5,1.0,0.25,4.00,0.25,...,0.5,0.125,2.00,0.125,2.00,0.500,0.25,0.5,0.5,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(fire, rock)",1.0,8.00,0.25,1.0,2.00,4.0,0.5,1.00,0.25,4.00,...,2.0,4.000,0.50,4.000,0.25,1.000,8.00,0.5,0.5,2.00
"(flying, ice)",1.0,0.50,1.00,0.5,4.00,1.0,2.0,1.00,4.00,2.00,...,1.0,2.000,0.25,8.000,1.00,0.125,1.00,2.0,4.0,2.00
"(bug, water)",1.0,2.00,1.00,0.5,2.00,1.0,1.0,1.00,4.00,0.50,...,2.0,1.000,1.00,2.000,1.00,2.000,0.50,1.0,2.0,1.00
"(bug, ground)",1.0,1.00,0.50,4.0,2.00,0.5,1.0,4.00,1.00,0.25,...,1.0,1.000,2.00,2.000,0.50,2.000,0.25,0.5,1.0,0.50


In [51]:
scores_df[scores_df < 2] = 0
scores_df

,normal,fire,water,electric,grass,ice,fighting,poison,ground,flying,...,"(psychic, ice)","(bug, poison)","(psychic, rock)","(bug, grass)","(normal, water)","(fire, rock)","(flying, ice)","(bug, water)","(bug, ground)","(bug, ghost)"
normal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fire,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,...,2.0,2.0,0.0,8.0,0.0,0.0,2.0,0.0,0.0,2.0
water,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0
electric,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0
grass,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(fire, rock)",0.0,8.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,4.0,...,2.0,4.0,0.0,4.0,0.0,0.0,8.0,0.0,0.0,2.0
"(flying, ice)",0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,4.0,2.0,...,0.0,2.0,0.0,8.0,0.0,0.0,0.0,2.0,4.0,2.0
"(bug, water)",0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,0.0,...,2.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0
"(bug, ground)",0.0,0.0,0.0,4.0,2.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0


In [52]:
with pd.option_context('display.max_rows', 50):
    #dis(pok_df.sort_values('total', ascending= False))
    pok_dft = pok_df.groupby(['type']).apply(lambda x: x.nlargest(1, "total")).reset_index(drop=True).sort_values('rank').reset_index().drop('index', axis=1)
    pok_dfat = pok_df.groupby(['type']).apply(lambda x: x.nlargest(1, "alttotal")).reset_index(drop=True).sort_values('alttotal', ascending=False).reset_index().drop('index', axis=1)
    dis(pok_dft[:50], pok_dfat[:50])

,rank,pokemon,hp,atk,def,satk,sdef,spd,type1,type2,total,alttotal,type
0,13,Slaking,150,160,100,95,65,100,normal,0,670,575,normal
1,21,Dragonite,91,134,95,100,100,80,dragon,flying,600,500,"(flying, dragon)"
2,22,Garchomp,108,130,95,80,85,102,dragon,ground,600,520,"(ground, dragon)"
3,28,Metagross,80,135,130,95,90,70,steel,psychic,600,505,"(steel, psychic)"
4,33,Tyranitar,100,134,110,95,100,61,rock,dark,600,505,"(rock, dark)"
5,46,Arcanine,90,110,80,100,80,95,fire,0,555,455,fire
6,47,Togekiss,85,50,95,120,115,80,normal,flying,545,495,"(flying, normal)"
7,49,Electivire,75,123,67,95,85,95,electric,0,540,445,electric
8,50,Gyarados,95,125,79,60,100,81,water,flying,540,480,"(flying, water)"
9,51,Kingdra,75,95,95,95,95,85,water,dragon,540,445,"(dragon, water)"


,rank,pokemon,hp,atk,def,satk,sdef,spd,type1,type2,total,alttotal,type
0,13,Slaking,150,160,100,95,65,100,normal,0,670,575,normal
1,22,Garchomp,108,130,95,80,85,102,dragon,ground,600,520,"(ground, dragon)"
2,33,Tyranitar,100,134,110,95,100,61,rock,dark,600,505,"(rock, dark)"
3,28,Metagross,80,135,130,95,90,70,steel,psychic,600,505,"(steel, psychic)"
4,21,Dragonite,91,134,95,100,100,80,dragon,flying,600,500,"(flying, dragon)"
5,111,Shuckle,20,10,230,10,230,5,bug,rock,505,495,"(bug, rock)"
6,47,Togekiss,85,50,95,120,115,80,normal,flying,545,495,"(flying, normal)"
7,59,Rhyperior,115,140,130,55,55,40,ground,rock,535,480,"(ground, rock)"
8,50,Gyarados,95,125,79,60,100,81,water,flying,540,480,"(flying, water)"
9,53,Milotic,95,60,79,100,125,81,water,0,540,480,water


In [57]:
def choose_poks(limit = 20, pok_df = pok_dfat):
    import collections

    first6 = pd.DataFrame([], index=range(limit), columns=range(6))
    saved_t = set()
    limit = limit

    for n in range(min(6,limit)):
        for k in range(min(6,limit)):
            if k <= n:
                dict1 = {}
                for comb in list(combinations(pok_df.iloc[:n+1]['type'],k+1)):
                    dict1[comb] = scores_df[scores_df.index.isin(comb)].max(axis=0).values
                first6.at[n,k] = [dict1]
    for n in tqdm_notebook(range(6,limit)):
        for k in range(6):
            if (k == 0) and (n <= limit - 6):
                dict1 = {}
                for comb in list(combinations(pok_df.iloc[:n+1]['type'],k+1)):
                    dict1[comb] = scores_df[scores_df.index.isin(comb)].max(axis=0).values
                first6.at[n,k] = [dict1]
            elif n - k < limit - 5:
                first6.at[n,k] = first6.at[n-1,k]
                last_comb = pok_df.iloc[n:n+1]['type']
                last_comb_value = scores_df[scores_df.index.isin(last_comb)].max(axis=0).values
                dict1 = {}
                for comb in list(first6.at[n-1,k-1][0].keys()):
                    new_comb = set(comb).union(set(last_comb))
                    update_value = np.maximum(first6.at[n-1,k-1][0][comb], last_comb_value)
                    if k == 5:
                        if (0 not in update_value or collections.Counter(update_value)[0] < 4) and new_comb not in saved_t:
                            saved_t.add(frozenset(new_comb))
                            #print(new_comb)
                    dict1[frozenset(new_comb)] = update_value
                try:
                    first6.at[n,k][0].update(dict1)
                except KeyError:
                    first6.at[n,k].update(dict1)
    return saved_t

In [58]:
def analyze_poks(saved_t):
    pok_d = {'comb': [], 'pokemon': [], '0score': [], '2score': [], 'total': [], 'alttotal': []}

    for comb in saved_t:
        if scores_df[scores_df.index.isin(comb)].max(axis=0).value_counts()[2]:
            test = scores_df[scores_df.index.isin(comb)].max(axis=0).value_counts()
            test.index = test.index.astype(int)
            if 0 not in test:
                zeroes = 0
                print('is_there')
            else:
                zeroes = test[0]
            pok_d['comb'].append(comb)
            pok_d['pokemon'].append([pok_dft[pok_dft['type'].isin(comb)]['pokemon']])
            pok_d['2score'].append(test[2])
            pok_d['0score'].append(zeroes)
            pok_d['total'].append(pok_dft[pok_dft['type'].isin(comb)]['total'].sum())
            pok_d['alttotal'].append(pok_dft[pok_dft['type'].isin(comb)]['alttotal'].sum())

    ranking = pd.DataFrame.from_dict(pok_d)
    return ranking

In [59]:
ranking_t = analyze_poks(choose_poks(limit = 50, pok_df = pok_dft))
ranking_at = analyze_poks(choose_poks(limit = 50, pok_df = pok_dfat))

<ipython-input-57-e9d3c366f338>:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n in tqdm_notebook(range(6,limit)):


In [311]:
with pd.option_context('display.max_colwidth', -1, 'display.max_rows', 65):
    #dis(ranking)
    dis(ranking_t.groupby(by=['0score']).apply(lambda x: x.nlargest(8,'total'))) 
    dis(ranking_at.groupby(by=['0score']).apply(lambda x: x.nlargest(2,'alttotal')))

comb  \
0score                                                                                                              
2      116  ((flying, dragon), (ground, grass), (fighting, fire), (steel, electric), (ice, water), (rock, dark))    
       13   ((ground, grass), (fighting, fire), (steel, electric), (normal, flying), (ice, water), (rock, dark))    
       36   ((ground, grass), (fighting, fire), (flying, water), (steel, electric), (ice, water), (rock, dark))     
       130  ((fighting, fire), (flying, water), (steel, electric), (ground, grass), (rock, dark), (ground, ice))    
       44   ((fighting, fire), (flying, water), (steel, electric), (ground, grass), (rock, dark), ice)              
       92   ((flying, dragon), (ground, grass), (fighting, fire), (steel, electric), dark, (ice, water))            
       5    ((fighting, fire), (poison, grass), (ground, flying), (steel, electric), (ice, water), (rock, dark))    
       33   ((ground, grass), (fighting, fire), (steel, electric), (normal, flying), (ice, water), dark)            
3      51   ((flying, dragon), (fighting, fire), (ground, dragon), (steel, electric), (ice, water), (rock, dark))   
       118  ((fighting, fire), (ground, dragon), (normal, flying), (ice, water), (rock, dark), (steel, electric))   
       91   ((fighting, fire), (flying, water), (ground, dragon), (steel, electric), (ice, water), (rock, dark))    
       105  ((flying, dragon), (steel, fighting), (ground, grass), (ice, water), (rock, dark), fire)                
       2    ((fighting, fire), (flying, water), (steel, electric), (ground, dragon), (rock, dark), (ground, ice))   
       81   ((fighting, fire), (flying, water), (steel, electric), (ground, dragon), (rock, dark), ice)             
       110  ((flying, dragon), (fighting, fire), (steel, electric), (ground, dragon), (ice, water), dark)           
       63   ((flying, dragon), (steel, fighting), (ground, grass), (ice, water), (rock, dark), electric)            

                                                                       pokemon  \
0score                                                                           
2      116  [[Dragonite, Tyranitar, Lapras, Magnezone, Infernape, Torterra]]     
       13   [[Tyranitar, Togekiss, Lapras, Magnezone, Infernape, Torterra]]      
       36   [[Tyranitar, Gyarados, Lapras, Magnezone, Infernape, Torterra]]      
       130  [[Tyranitar, Gyarados, Magnezone, Infernape, Mamoswine, Torterra]]   
       44   [[Tyranitar, Gyarados, Magnezone, Infernape, Glaceon, Torterra]]     
       92   [[Dragonite, Lapras, Magnezone, Infernape, Torterra, Umbreon]]       
       5    [[Tyranitar, Lapras, Magnezone, Infernape, Venusaur, Gliscor]]       
       33   [[Togekiss, Lapras, Magnezone, Infernape, Torterra, Umbreon]]        
3      51   [[Dragonite, Garchomp, Tyranitar, Lapras, Magnezone, Infernape]]     
       118  [[Garchomp, Tyranitar, Togekiss, Lapras, Magnezone, Infernape]]      
       91   [[Garchomp, Tyranitar, Gyarados, Lapras, Magnezone, Infernape]]      
       105  [[Dragonite, Tyranitar, Arcanine, Lapras, Lucario, Torterra]]        
       2    [[Garchomp, Tyranitar, Gyarados, Magnezone, Infernape, Mamoswine]]   
       81   [[Garchomp, Tyranitar, Gyarados, Magnezone, Infernape, Glaceon]]     
       110  [[Dragonite, Garchomp, Lapras, Magnezone, Infernape, Umbreon]]       
       63   [[Dragonite, Tyranitar, Electivire, Lapras, Lucario, Torterra]]      

            0score  2score  total  alttotal  
0score                                       
2      116  2       29      3329   2800      
       13   2       33      3274   2795      
       36   2       30      3269   2780      
       130  2       29      3264   2790      
       44   2       28      3259   2795      
       92   2       32      3254   2760      
       5    2       25      3239   2758      
       33   2       36      3199   2755      
3      51   3       30      3404   2870      
       118  3      

comb  \
0score                                                                                                        
2      76   ((flying, water), (steel, electric), (ground, grass), (bug, fighting), (ice, dark), fire)         
       145  ((flying, water), (steel, electric), (psychic, fighting), (ground, grass), (ice, dark), fire)     
3      57   ((flying, water), (steel, electric), (ground, grass), normal, (bug, fighting), (ice, dark))       
       206  ((flying, water), (steel, electric), (psychic, fighting), (ground, grass), normal, (ice, dark))   

                                                                    pokemon  \
0score                                                                        
2      76   [[Arcanine, Gyarados, Magnezone, Torterra, Weavile, Heracross]]   
       145  [[Arcanine, Gyarados, Magnezone, Torterra, Gallade, Weavile]]     
3      57   [[Slaking, Gyarados, Magnezone, Torterra, Weavile, Heracross]]    
       206  [[Slaking, Gyarados, Magnezone, Torterra, Gallade, Weavile]]      

            0score  2score  total  alttotal  
0score                                       
2      76   2       30      3165   2775      
       145  2       31      3183   2768      
3      57   3       32      3280   2895      
       206  3       34      3298   2888

In [40]:
pok_dft.iloc[n:n+1]['type']

5    fire
Name: type, dtype: object

In [35]:
first6 = pd.DataFrame()
limit = 7

for n in range(limit):
    for k in range(limit):
        if n==0 or k==0:
            print(n,k)
            dict1 = {}
            for comb in list(combinations(pok_dft.iloc[:n+1]['type'],k+1)):
                dict1[comb] = scores_df[scores_df.index.isin(comb)].max(axis=0).values
            first6.at[n,k] = [dict1]

first6

0 0
0 1
0 2
0 3
0 4
0 5
0 6
1 0
2 0
3 0
4 0
5 0
6 0


,0,1,2,3,4,5,6
0,"{('normal',): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",{},{},{},{},{},{}
1,"[{('normal',): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,NaN,NaN,NaN,NaN,NaN
2,"[{('normal',): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,NaN,NaN,NaN,NaN,NaN
3,"[{('normal',): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,NaN,NaN,NaN,NaN,NaN
4,"[{('normal',): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,NaN,NaN,NaN,NaN,NaN
5,"[{('normal',): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,NaN,NaN,NaN,NaN,NaN
6,"[{('normal',): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,NaN,NaN,NaN,NaN,NaN


In [314]:
after6 = pd.DataFrame()
r = 3
for n in range(6,r+6):
    for k in range(1,r+1):
        if k <= n-6:
            dict1 = {}
            print(n,k)
            for comb in list(combinations(pok_dft.iloc[6:n+k-1]['type'],k)):
                dict1[comb] = scores_df[scores_df.index.isin(comb)].max(axis=0).values
                print(comb)
            after6.at[n-6,k] = [dict1]

after6


TypeError: 'int' object is not callable

In [317]:
first6 = pd.DataFrame()
for n in range(1,7):
    for k in range(1,7):
        if k <= n:
            dict1 = {}
            for comb in list(combinations(pok_dft.iloc[:n]['type'],k)):
                dict1[comb] = scores_df[scores_df.index.isin(comb)].max(axis=0).values
            first6.at[n-1,k-1] = [dict1]

first6

TypeError: 'int' object is not callable

In [129]:
no_zeroes_t = set()
# one_zeroes_t = set()
no_twos_t = set()


for i in tqdm_notebook(range(23,28)):
    slice = pok_dft.iloc[:i,:]
    combs = list(combinations(slice.type, 6))
    for comb in combs:
        scores = scores_df[scores_df.index.isin(comb)].max(axis=0).values
        if 2 not in scores:
            no_twos_t.add(comb)
            print(pok_dft[pok_dft['type'].isin(comb)]['pokemon'], collections.Counter(scores))
        elif 0 not in scores:
            no_zeroes_t.add(comb)
            print(pok_dft[pok_dft['type'].isin(comb)]['pokemon'], collections.Counter(scores))
        elif collections.Counter(scores)[0] == 1:
            one_zeroes_t.add(comb)
            print(pok_dft[pok_dft['type'].isin(comb)]['pokemon'], collections.Counter(scores))

len(no_zeroes_t), len(no_twos_t), len(one_zeroes_t)

<ipython-input-129-007e7ac9333b>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(23,28)):


2      Garchomp
4     Tyranitar
6      Togekiss
8      Gyarados
13    Magnezone
16    Infernape
Name: pokemon, dtype: object Counter({4.0: 42, 2.0: 32, 8.0: 13, 16.0: 1, 0.0: 1})
2      Garchomp
4     Tyranitar
6      Togekiss
9       Kingdra
13    Magnezone
16    Infernape
Name: pokemon, dtype: object Counter({4.0: 38, 2.0: 36, 8.0: 13, 16.0: 1, 0.0: 1})
2      Garchomp
4     Tyranitar
6      Togekiss
10      Milotic
13    Magnezone
16    Infernape
Name: pokemon, dtype: object Counter({4.0: 39, 2.0: 36, 8.0: 12, 16.0: 1, 0.0: 1})
2      Garchomp
4     Tyranitar
6      Togekiss
12       Lapras
13    Magnezone
16    Infernape
Name: pokemon, dtype: object Counter({4.0: 40, 2.0: 35, 8.0: 12, 16.0: 1, 0.0: 1})
2      Garchomp
4     Tyranitar
6      Togekiss
8      Gyarados
13    Magnezone
16    Infernape
Name: pokemon, dtype: object Counter({4.0: 42, 2.0: 32, 8.0: 13, 16.0: 1, 0.0: 1})
2      Garchomp
4     Tyranitar
6      Togekiss
9       Kingdra
13    Magnezone
16    Infernape
Name: pok

(8, 0, 26)

In [130]:
no_zeroes_at = set()
# one_zeroes_at = set()
no_twos_at = set()

for i in tqdm_notebook(range(23,28)):
    slice = pok_dfat.iloc[:i,:]
    combs = list(combinations(slice.type, 6))
    for comb in combs:
        scores = scores_df[scores_df.index.isin(comb)].max(axis=0).values
        if 2 not in scores:
            no_twos_at.add(comb)
            print(pok_dft[pok_dft['type'].isin(comb)]['pokemon'], collections.Counter(scores))
        elif 0 not in scores:
            no_zeroes_at.add(comb)
            print(pok_dft[pok_dft['type'].isin(comb)]['pokemon'], collections.Counter(scores))
        elif collections.Counter(scores)[0] == 1:
            one_zeroes_at.add(comb)
            print(pok_dft[pok_dft['type'].isin(comb)]['pokemon'], collections.Counter(scores))

len(no_zeroes_at), len(no_twos_at), len(one_zeroes_at)

<ipython-input-130-9b9623828882>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(23,28)):


6      Togekiss
13    Magnezone
15    Tangrowth
36      Gliscor
38      Weavile
45    Heracross
Name: pokemon, dtype: object Counter({4.0: 45, 2.0: 32, 8.0: 10, 16.0: 1, 0.0: 1})
6      Togekiss
13    Magnezone
15    Tangrowth
36      Gliscor
38      Weavile
45    Heracross
Name: pokemon, dtype: object Counter({4.0: 45, 2.0: 32, 8.0: 10, 16.0: 1, 0.0: 1})
6      Togekiss
13    Magnezone
15    Tangrowth
36      Gliscor
38      Weavile
45    Heracross
Name: pokemon, dtype: object Counter({4.0: 45, 2.0: 32, 8.0: 10, 16.0: 1, 0.0: 1})
6      Togekiss
13    Magnezone
15    Tangrowth
36      Gliscor
38      Weavile
45    Heracross
Name: pokemon, dtype: object Counter({4.0: 45, 2.0: 32, 8.0: 10, 16.0: 1, 0.0: 1})
2      Garchomp
5      Arcanine
6      Togekiss
13    Magnezone
38      Weavile
45    Heracross
Name: pokemon, dtype: object Counter({4.0: 38, 2.0: 37, 8.0: 12, 16.0: 1, 0.0: 1})
6      Togekiss
13    Magnezone
15    Tangrowth
36      Gliscor
38      Weavile
45    Heracross
Name: pok

(0, 0, 2)

In [105]:
for comb in one_zeroes_t:
    if scores_df[scores_df.index.isin(comb)].max(axis=0).value_counts()[2] < 40:
        dis(scores_df.columns[scores_df[scores_df.index.isin(comb)].max(axis= 0) == 16])
        test = scores_df[scores_df.index.isin(comb)].max(axis=0).value_counts()
        test.index = test.index.astype(int)
        dis(test)

Index([('bug', 'fighting')], dtype='object')

4     42
2     32
8     14
0      1
16     1
dtype: int64

Index([('bug', 'fighting')], dtype='object')

4     38
2     36
8     14
0      1
16     1
dtype: int64

Index([('bug', 'fighting')], dtype='object')

4     40
2     35
8     13
0      1
16     1
dtype: int64

Index([('bug', 'fighting')], dtype='object')

4     39
2     36
8     13
0      1
16     1
dtype: int64

In [104]:
with pd.option_context('display.max_rows', 90):
    dis(type_chart_joined[frozenset({'water', 'ground'})])

normal                 1.0
fire                   0.5
water                  1.0
electric               0.0
grass                  4.0
ice                    1.0
fighting               1.0
poison                 0.5
ground                 1.0
flying                 1.0
psychic                1.0
bug                    1.0
rock                   0.5
ghost                  1.0
dragon                 1.0
dark                   1.0
steel                  0.5
fairy                  1.0
(dragon, flying)       1.0
(dragon, ground)       1.0
(psychic, steel)       1.0
(rock, dark)           1.0
(fairy, flying)        1.0
(water, flying)        1.0
(dragon, water)        1.0
(poison, flying)       1.0
(ice, water)           1.0
(steel, electric)      0.5
(rock, ground)         1.0
(fire, flying)         1.0
(fire, fighting)       1.0
(steel, rock)          0.5
(steel, water)         1.0
(ice, ground)          1.0
(steel, fighting)      1.0
(grass, ground)        4.0
(grass, poison)        4.0
(

In [68]:
pok_d = {'comb': [], 'pokemon': [], '2score': [], 'total': [], 'alttotal': []}

for comb in one_zeroes_t:
    if scores_df[scores_df.index.isin(comb)].max(axis=0).value_counts()[2]:
        test = scores_df[scores_df.index.isin(comb)].max(axis=0).value_counts()
        test.index = test.index.astype(int)
        #dis(test.sort_index(), pok_dft[pok_dft['type'].isin(comb)]['pokemon'])
        pok_d['comb'].append(comb)
        pok_d['pokemon'].append([pok_dft[pok_dft['type'].isin(comb)]['pokemon']])
        pok_d['2score'].append(test[2])
        pok_d['total'].append(pok_dft[pok_dft['type'].isin(comb)]['total'].sum())
        pok_d['alttotal'].append(pok_dft[pok_dft['type'].isin(comb)]['alttotal'].sum())

ranking = pd.DataFrame.from_dict(pok_d)

In [70]:
with pd.option_context('display.max_colwidth', -1):
    dis(ranking)
    dis(ranking.sort_values('2score').groupby(by='2score').apply(lambda x: x.nlargest(1,'total')))
    ranking.sort_values('total', ascending=False)

,comb,pokemon,2score,total,alttotal
0,"((dragon, ground), (rock, dark), (fairy, flying), (water, flying), (steel, electric), (fire, fighting))","[[Garchomp, Tyranitar, Togekiss, Gyarados, Magnezone, Infernape]]",32,3354,2895
1,"((dragon, ground), (rock, dark), (fairy, flying), (dragon, water), (steel, electric), (fire, fighting))","[[Garchomp, Tyranitar, Togekiss, Kingdra, Magnezone, Infernape]]",36,3354,2860
2,"((dragon, ground), (rock, dark), (fairy, flying), (ice, water), (steel, electric), (fire, fighting))","[[Garchomp, Tyranitar, Togekiss, Lapras, Magnezone, Infernape]]",35,3349,2865
3,"((dragon, ground), (rock, dark), (fairy, flying), water, (steel, electric), (fire, fighting))","[[Garchomp, Tyranitar, Togekiss, Milotic, Magnezone, Infernape]]",36,3354,2895


,,comb,pokemon,2score,total,alttotal
2score,,,,,,
32,0,"((dragon, ground), (rock, dark), (fairy, flying), (water, flying), (steel, electric), (fire, fighting))","[[Garchomp, Tyranitar, Togekiss, Gyarados, Magnezone, Infernape]]",32,3354,2895
35,2,"((dragon, ground), (rock, dark), (fairy, flying), (ice, water), (steel, electric), (fire, fighting))","[[Garchomp, Tyranitar, Togekiss, Lapras, Magnezone, Infernape]]",35,3349,2865
36,1,"((dragon, ground), (rock, dark), (fairy, flying), (dragon, water), (steel, electric), (fire, fighting))","[[Garchomp, Tyranitar, Togekiss, Kingdra, Magnezone, Infernape]]",36,3354,2860


In [21]:
for comb in ranking['comb'][:2]:
    type_df = scores_df[scores_df.index.isin(comb)]
    #dis(type_df.columns[type_df[type_df[type_df.max()[type_df.max() == 2].index] == 2].count() < 2].shape[0])
    dis(type_df[type_df.max()[type_df.max() == 2].index].sum()/2)


normal                 1.0
(dragon, flying)       2.0
(psychic, steel)       1.0
(dragon, water)        1.0
(ice, water)           1.0
(fire, fighting)       2.0
(steel, water)         2.0
(ice, ground)          1.0
(steel, fighting)      2.0
(grass, poison)        1.0
(grass, psychic)       1.0
(ghost, electric)      2.0
(psychic, water)       2.0
(psychic, fighting)    1.0
(poison, water)        2.0
(flying, ground)       1.0
(steel, ground)        1.0
(psychic, ground)      2.0
(poison, dark)         1.0
(water, fighting)      2.0
(poison, ground)       1.0
(poison, fighting)     1.0
(dark, ghost)          1.0
(grass, water)         1.0
(water, dark)          3.0
(grass, flying)        1.0
(bug, poison)          1.0
(fairy, water)         2.0
(water, normal)        2.0
(steel, fairy)         2.0
(bug, water)           2.0
(bug, ghost)           2.0
dtype: float64

In [22]:
for 

SyntaxError: invalid syntax (<ipython-input-22-eaf083cb72bb>, line 1)

In [ ]:
dict1 = {'df': [1,2], 'dsf':[12,31]}
pd.DataFrame.from_dict(dict1)

,df,dsf
0,1,12
1,2,31


In [ ]:
for i in range(6,7):
    with pd.option_context('display.max_rows', 100, 'display.max_columns', 90):
        dis(pok_df.iloc[:i,:]['type'])
        dis(scores_df[scores_df.index.isin(pok_df.iloc[:i,:]['type'])].max(axis=0))

0              normal
1    (flying, dragon)
2    (ground, dragon)
3    (steel, psychic)
4    (flying, dragon)
5        (rock, dark)
Name: type, dtype: object

normal                 2.0
fire                   4.0
water                  2.0
electric               4.0
grass                  8.0
ice                    4.0
fighting               4.0
poison                 4.0
ground                 4.0
flying                 4.0
psychic                4.0
bug                    4.0
rock                   4.0
ghost                  4.0
dragon                 2.0
dark                   2.0
steel                  2.0
fairy                  4.0
(flying, dragon)       2.0
(ground, dragon)       0.0
(steel, psychic)       2.0
(rock, dark)           2.0
(fairy, flying)        4.0
(water, flying)        0.0
(water, dragon)        0.0
(flying, poison)       4.0
(ice, water)           0.0
(electric, steel)      4.0
(rock, ground)         2.0
(water, ground)        2.0
(flying, fire)         8.0
(fighting, fire)       4.0
(steel, rock)          4.0
(steel, water)         2.0
(ice, ground)          0.0
(steel, fighting)      2.0
(ground, grass)        8.0
(

In [ ]:
pok_df['score'] = np.nan
for i,pok in enumerate(pok_df[:6].itertuples()):
    pok_df.at[i, 'score'] = frozenlist(scores_df.loc[pok.type,:].tolist())

pok_df

NameError: name 'frozenlist' is not defined